## CONFIG & HELPERS


In [ ]:
# Install dependencies
%pip install --quiet torch torchvision torchaudio tensorboard scikit-learn matplotlib pandas

import os, math, random, numpy as np, pandas as pd
import torch, torchvision, torch.nn as nn, torch.nn.functional as F
from pathlib import Path
from typing import Dict, Tuple

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

def get_output_shape(model: nn.Module, image_dim: Tuple[int,int,int,int]):
    model.eval()
    with torch.no_grad():
        x = torch.randn(*image_dim, device=device)
        out = model(x)
        return tuple(out.shape)

def xyxy_to_valid(b: torch.Tensor) -> torch.Tensor:
    b = b.float().view(-1,4)
    x1 = torch.minimum(b[:,0], b[:,2])
    y1 = torch.minimum(b[:,1], b[:,3])
    x2 = torch.maximum(b[:,0], b[:,2])
    y2 = torch.maximum(b[:,1], b[:,3])
    return torch.stack([x1,y1,x2,y2], dim=1)

def clamp_01(b: torch.Tensor) -> torch.Tensor:
    return b.clamp(0.0, 1.0)


## DATA PREPARATION & EXPLORATION


In [ ]:
# Load training data
train_path = Path('fa-ii-2025-ii-object-localization/train.csv')
train_df = pd.read_csv(train_path)
print('Train samples:', len(train_df))
display(train_df.head())

# Class distribution
print(train_df['class'].value_counts())
